In [1]:
### Q1.
### Load packages and preprocess data using professor's Starter

## Load packages and data
using JSON
using JuMP, Gurobi, Polynomials

instance="2020-12-23.json"

## Define function for solving Unit commitment proplem
function solve_uc(data_file, option, nlins)

########### This is just data reading, no need to edit ######
    data = JSON.parsefile(data_file)

    T = data["time_periods"]
    thermal_gens = keys(data["thermal_generators"])
    renewable_gens = keys(data["renewable_generators"])
    periods = 1:T

    demand = [data["demand"][t] for t in 1:T]
    gen_pwl_points = Dict(g => 1:length(gen["piecewise_production"]) for (g,gen) in data["thermal_generators"])
    
    min_down_time=Dict()
    min_up_time=Dict()
    min_power=Dict()
    max_power=Dict()
    cost_sq_coef=Dict()
    cost_lin_coef=Dict()
    cost_const=Dict()
    for (g,gen) in data["thermal_generators"]
        min_down_time[g]= gen["time_down_minimum"]
        min_up_time[g]=gen["time_up_minimum"]
        min_power[g]=gen["power_output_minimum"]
        max_power[g]=gen["power_output_maximum"]
        xpts=zeros(length(gen_pwl_points[g]))
        ypts=zeros(length(gen_pwl_points[g]))
        for l in  gen_pwl_points[g]
            xpts[l]=gen["piecewise_production"][l]["mw"]
            ypts[l]=gen["piecewise_production"][l]["cost"]
        end
        if length(gen_pwl_points[g]) > 2
           f = fit(xpts,ypts,2)
            if (f.coeffs[3]> 0.000001 || f.coeffs[3] < -0.000001)
                cost_sq_coef[g]=f.coeffs[3]
            else
                cost_sq_coef[g]=0.0
            end
            cost_lin_coef[g]=f.coeffs[2]
            cost_const[g]=f.coeffs[1]
        elseif length(gen_pwl_points[g]) == 2
            cost_sq_coef[g]=0.0
            cost_lin_coef[g]=(ypts[2]-ypts[1])/(xpts[2]-xpts[1])
            cost_const[g]=ypts[1]-cost_lin_coef[g]*xpts[1]
        else
            cost_sq_coef[g]=0.0
            cost_lin_coef[g]=ypts[1]/xpts[1]
            cost_const[g]=0.0
        end
    end

    rgen_min=Dict()
    rgen_max=Dict()

    for (g,gen) in data["renewable_generators"]
        for t in 1:T
            rgen_min[g,t]=gen["power_output_minimum"][t]
            rgen_max[g,t]=gen["power_output_maximum"][t]
        end
    end
###### End of data reading ######

    # Define the set which contains all of generators
    total_gens = union(thermal_gens, renewable_gens)
    
    # Create the model set optimizer and time limit
    m = Model()
    set_optimizer(m, Gurobi.Optimizer) 
    set_time_limit_sec(m,200)
    
    # Declare all the decision variables
    @variable(m, x[thermal_gens, 1:T], Bin)
    @variable(m, p[total_gens, 1:T])
    
    # Write all the constraints and additional variables
    @constraint(m, ther_power_const_lhs[g in thermal_gens, t in 1:T], min_power[g]*x[g,t] <= p[g,t])
    @constraint(m, ther_power_const_rhs[g in thermal_gens, t in 1:T], p[g,t] <= max_power[g]*x[g,t])
 
    @constraint(m, demand[t in 1:T], sum(p[g, t] for g in total_gens) >= demand[t])

    @constraint(m, ren_power_const_lhs[g in renewable_gens, t in 1:T], rgen_min[g,t] <= p[g,t])
    @constraint(m, ren_power_const_rhs[g in renewable_gens, t in 1:T], p[g,t] <= rgen_max[g,t])

    @variable(m, w[thermal_gens, 1:T], Bin)
    @variable(m, v[thermal_gens, 1:T], Bin)

    @constraint(m, turn_on_off_0[g in thermal_gens], x[g,1] == w[g,1] - v[g,1])
    @constraint(m, turn_on_off[g in thermal_gens, t in 2:T], x[g,t] == x[g,t-1] + w[g,t] - v[g,t])
        
    @constraint(m, min_up_time[g in thermal_gens, t in 1:T], sum(w[g,s] for s in max(t-min_up_time[g]+1,1):t) <= x[g,t])
    @constraint(m, min_down_time[g in thermal_gens, t in 1:T], sum(v[g,s] for s in max(t-min_down_time[g]+1,1):t) <= 1 - x[g,t])

    # Solve the formulation with quadratic objective
    if option == 1
        
        # Define the objective
        @objective(m, Min, sum(cost_sq_coef[g]*(p[g,t])^2+cost_lin_coef[g]*p[g,t]+cost_const[g]*x[g,t] for g in thermal_gens, t in 1:T))

        # Solve the model
        optimize!(m)

        # Print required results
        println("---------------------------------------")

        # Print Solution time
        println("Solution time: ", solve_time(m))

        # Print Best objective of a feasible solution
        println("Best objective: ", objective_value(m))

        # Print Best bound
        println("Best bound: ", objective_bound(m))

        # Print Ending gap
        println("Ending gap: ", (objective_value(m)-objective_bound(m))/objective_value(m)*100, "%")
        
        # Print Number of explored nodes
        println("Number of branch-and-bound nodes explored: ", node_count(m))

    # Solve formulation with separable quadratic terms included in constraints        
    elseif option == 2
        
        # Define the objective and additional variables, constraints
        @variable(m, y[thermal_gens, 1:T])
        
        @objective(m, Min, sum(y[g,t] for g in thermal_gens, t in 1:T))
        
        @constraint(m, objective[g in thermal_gens, t in 1:T], y[g,t] >= cost_sq_coef[g]*(p[g,t])^2+cost_lin_coef[g]*p[g,t]+cost_const[g]*x[g,t])
        
        # Solve the model
        optimize!(m)

        # Print required results
        println("---------------------------------------")

        # Print Solution time
        println("Solution time: ", solve_time(m))

        # Print Best objective of a feasible solution
        println("Best objective: ", objective_value(m))

        # Print Best bound
        println("Best bound: ", objective_bound(m))

        # Print Ending gap
        println("Ending gap: ", (objective_value(m)-objective_bound(m))/objective_value(m)*100, "%")
        
        # Print Number of explored nodes
        println("Number of branch-and-bound nodes explored: ", node_count(m))

    # Solve a piecewise linear relaxation with cuts
    elseif option == 3
        
        # Define the objective and additional variables, constraints
        @variable(m, y[thermal_gens, 1:T])
        
        @objective(m, Min, sum(y[g,t] for g in thermal_gens, t in 1:T))
    
        linearizations = 1:nlins
        
        @constraint(m, objective[g in thermal_gens, t in 1:T, i in linearizations],
            y[g,t] >= (cost_const[g]-cost_sq_coef[g]*(min_power[g] + (max_power[g] - min_power[g])*(i-1)/(nlins-1))^2)*x[g,t]
            + (2*cost_sq_coef[g]*(min_power[g] + (max_power[g] - min_power[g])*(i-1)/(nlins-1)) + cost_lin_coef[g])*p[g,t])

        # Solve the model
        optimize!(m)
        x_value = value.(x)
        p_value = value.(p)

        # For calculate the real value
        real_value = sum(cost_sq_coef[g]*(p_value[g,t])^2+cost_lin_coef[g]*p_value[g,t]+cost_const[g]*x_value[g,t] for g in thermal_gens, t in 1:T)
        
        # Print required results
        println("---------------------------------------")

        # Print Solution time
        println("Solution time: ", solve_time(m))

        # Print Best objective of a feasible solution
        println("Best objective: ", real_value)

        # Print Number of explored nodes
        println("Number of branch-and-bound nodes explored: ", node_count(m))
    
    end
end


solve_uc (generic function with 1 method)

In [2]:
### Solve the problem using option 1
solve_uc(instance,1,0)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 200
Set parameter TimeLimit to value 200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 25344 rows, 17904 columns and 81662 nonzeros
Model fingerprint: 0x8c366557
Model has 3504 quadratic objective terms
Variable types: 7392 continuous, 10512 integer (10512 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [6e-01, 6e+03]
  QObjective range [3e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 5e+03]
Presolve removed 8555 ro

In [3]:
### Solve the problem using option 2
solve_uc(instance,2,0)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 200
Set parameter TimeLimit to value 200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 25344 rows, 21408 columns and 81662 nonzeros
Model fingerprint: 0x7d9f48cc
Model has 3504 quadratic constraints
Variable types: 10896 continuous, 10512 integer (10512 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  QMatrix range    [1e-03, 5e+00]
  QLMatrix range   [6e-01, 6e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 

In [2]:
### Solve the problem using option 3 with 3 linearizations
solve_uc(instance,3,3) 

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 200
Set parameter TimeLimit to value 200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 35856 rows, 21408 columns and 113198 nonzeros
Model fingerprint: 0x5a215492
Variable types: 10896 continuous, 10512 integer (10512 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 5e+03]
Presolve removed 8380 rows and 4349 columns
Presolve time: 0.67s
Presolved: 27476 rows, 17059 col

In [3]:
### Solve the problem using option 3 with 5 linearizations
solve_uc(instance,3,5)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 200
Set parameter TimeLimit to value 200
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 42864 rows, 21408 columns and 134222 nonzeros
Model fingerprint: 0x300131d5
Variable types: 10896 continuous, 10512 integer (10512 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 5e+03]
Presolve removed 8386 rows and 4349 columns
Presolve time: 0.51s
Presolved: 34478 rows, 17059 col

In [6]:
### Q2.
### Load packages and preprocess data using professor's Starter

## Load packages and data
using JuMP, Plots, Gurobi

filename="ad-data.txt"

## Preprocess data
strdata=split(read(filename, String))
data = [parse(Float64,s) for s in strdata]
np=Int(data[1])
budget=data[2]
nprecs=Int(data[3])
fixcost=zeros(Float64,np)
u=zeros(Float64,np)

# Functional form: s*exp(ax+b)/(1+ exp(ax+b))
varcost=zeros(Float64,np)
s = zeros(Float64,np)
a = zeros(Float64,np)
b = zeros(Float64,np)
curind=4
for i in 1:np
    fixcost[i]=data[curind]
    curind+=1
    varcost[i]=data[curind]
    curind+=1
    u[i]=data[curind]
    curind+=1
    s[i]=data[curind]
    curind+=1
    a[i]=data[curind]
    curind+=1
    b[i]=data[curind]
    curind+=1
end
precs=[]
for k in 1:nprecs
    push!(precs,(Int(data[curind]),Int(data[curind+1])))
    curind=curind+2
end

# Evaluates the ith product function at point x
function feval(i, x)
    return s[i]*exp(a[i]*x+b[i])/(1+ exp(a[i]*x + b[i]))
end

# Evaluates the ith product function at point x considering y variable
function feval_2(i, x, y)
    return feval(i, x)-(1-y)*feval(i, 0)
end

### Define function for solving advertising allocation proplem
function solvelinapprox(nlins,opt)

    ## Create the model set optimizer and time limit
    m = Model()
    set_optimizer(m, Gurobi.Optimizer) 
    set_time_limit_sec(m,300)
    
    ## Declare all the decision variables
    @variable(m, x[1:np])
    @variable(m, y[1:np], Bin)
    @variable(m, t[1:np])
    @variable(m, z[1:np,1:nlins], Bin)
    @variable(m, v[1:np], Bin)
    @variable(m, w[1:np,1:nlins])
    
    ## Define the objective
    @objective(m, Max, sum(t[i] for i in 1:np))

    ## Write all the constraints
    @constraint(m, total_budget, sum(varcost[i]x[i]+fixcost[i]y[i] for i in 1:np) <= budget)
    @constraint(m, upper_limit[i in 1:np], x[i] <= u[i]y[i])
    @constraint(m, pre_rel[k in 1:nprecs], y[(precs[k][1])] <= y[(precs[k][2])])

    ## Not using indicator strengthened form
    if opt == 1
        # t_i <= sum(m_ik*w_ik + n_ik*z_ik) - (1-v_i)*g_i(0,y_i) 
        @constraint(m, objective[i in 1:np],
            t[i] <= sum(((feval_2(i,u[i]*k/nlins,y[i])-feval_2(i,u[i]*(k-1)/nlins,y[i]))/u[i]*nlins)*w[i,k] 
                + (feval_2(i,u[i]*(k-1)/nlins,y[i])-((feval_2(i,u[i]*k/nlins,y[i])-feval_2(i,u[i]*(k-1)/nlins,y[i]))/u[i]*nlins)*u[i]*(k-1)/nlins)*z[i,k]
                for k in 1:nlins)-(1-v[i])*feval_2(i,0,y[i]))
        # sum(z_ik) = 1
        @constraint(m, sum_z[i in 1:np], sum(z[i,k] for k in 1:nlins) == 1)
        # sum(w_ik) = x_i
        @constraint(m, sum_w[i in 1:np], sum(w[i,k] for k in 1:nlins) == x[i])
        # For a_ik(breakpoints), a_ik*z_ik <= w_ik <= a_{i(k+1)}*z_ik
        @constraint(m, inter_w_lhs[i in 1:np, k in 1:nlins], u[i]*(k-1)/nlins*z[i,k] <= w[i,k])
        @constraint(m, inter_w_rhs[i in 1:np, k in 1:nlins], w[i,k] <= u[i]*k/nlins*z[i,k])
        # x_i <= u_i*v_i
        @constraint(m, inter_x[i in 1:np], x[i] <= u[i]*v[i])
        
    ## Using indicator strengthened form
    elseif opt == 2
        # t_i <= sum(m_ik*w_ik + n_ik*z_ik)
        @constraint(m, objective[i in 1:np],
            t[i] <= sum(((feval_2(i,u[i]*k/nlins,y[i])-feval_2(i,u[i]*(k-1)/nlins,y[i]))/u[i]*nlins)*w[i,k] 
                + (feval_2(i,u[i]*(k-1)/nlins,y[i])-((feval_2(i,u[i]*k/nlins,y[i])-feval_2(i,u[i]*(k-1)/nlins,y[i]))/u[i]*nlins)*u[i]*(k-1)/nlins)*z[i,k]
                for k in 1:nlins))
        # sum(z_ik) = v_i
        @constraint(m, sum_z[i in 1:np], sum(z[i,k] for k in 1:nlins) == v[i])
        # sum(w_ik) = x_i
        @constraint(m, sum_w[i in 1:np], sum(w[i,k] for k in 1:nlins) == x[i])
        # For a_ik(breakpoints), a_ik*z_ik <= w_ik <= a_{i(k+1)}*z_ik
        @constraint(m, inter_w_lhs[i in 1:np, k in 1:nlins], u[i]*(k-1)/nlins*z[i,k] <= w[i,k])
        @constraint(m, inter_w_rhs[i in 1:np, k in 1:nlins], w[i,k] <= u[i]*k/nlins*z[i,k])

    end

    ## Solve the model
    optimize!(m)
    x_value = value.(x)
    y_value = value.(y)

    ## For calculate the real value
    real_value = sum(feval(i,x_value[i])-(1-y_value[i])*feval(i,0) for i in 1:np)

    ## Print required results
    println("---------------------------------------")

    ## Print Solution time
    println("Solution time: ", solve_time(m))

    ## Print Number of explored nodes
    println("Number of branch-and-bound nodes: ", node_count(m))

    ## Print Best objective of a feasible solution
    println("Best objective by the solver: ", objective_value(m))

    ## Print Best bound
    println("Best bound by the solver: ", objective_bound(m))

    ## Print True objective
    println("True objective value is ", real_value)    
end

solvelinapprox (generic function with 1 method)

In [7]:
### Solve the problem using option 1 with break points value 5
solvelinapprox(5,1)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 18936 rows, 14000 columns and 45644 nonzeros
Model fingerprint: 0x8174196a
Model has 1000 quadratic constraints
Variable types: 7000 continuous, 7000 integer (7000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [1e-03, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 8e+

In [8]:
### Solve the problem using option 1 with break points value 10
solvelinapprox(10,1)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 28936 rows, 24000 columns and 75644 nonzeros
Model fingerprint: 0xdc5673e2
Model has 1000 quadratic constraints
Variable types: 12000 continuous, 12000 integer (12000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [1e-03, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 

In [9]:
### Solve the problem using option 1 with break points value 20
solvelinapprox(20,1)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 48936 rows, 44000 columns and 135644 nonzeros
Model fingerprint: 0xd3d98959
Model has 1000 quadratic constraints
Variable types: 22000 continuous, 22000 integer (22000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [5e-04, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00,

In [10]:
### Solve the problem using option 2 with break points value 5
solvelinapprox(5,2)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 17936 rows, 14000 columns and 44644 nonzeros
Model fingerprint: 0x4b271b95
Model has 1000 quadratic constraints
Variable types: 7000 continuous, 7000 integer (7000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [3e-03, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 8e+

In [11]:
### Solve the problem using option 2 with break points value 10
solvelinapprox(10,2)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 27936 rows, 24000 columns and 74644 nonzeros
Model fingerprint: 0xbee6fc7c
Model has 1000 quadratic constraints
Variable types: 12000 continuous, 12000 integer (12000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [3e-03, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 

In [12]:
### Solve the problem using option 2 with break points value 20
solvelinapprox(20,2)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498709
Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Set parameter TimeLimit to value 300
Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2498709 - for non-commercial use only - registered to hy___@wisc.edu
Optimize a model with 47936 rows, 44000 columns and 134644 nonzeros
Model fingerprint: 0x9d00feaa
Model has 1000 quadratic constraints
Variable types: 22000 continuous, 22000 integer (22000 binary)
Coefficient statistics:
  Matrix range     [2e-02, 6e+00]
  QMatrix range    [1e-03, 1e-01]
  QLMatrix range   [5e-04, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02,